In [1]:
!pip install --upgrade azureml-sdk azureml-widgets

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.19.0)
Requirement already up-to-date: azureml-widgets in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.19.0)
  Using cached pyOpenSSL-19.1.0-py2.py3-none-any.whl (53 kB)


ERROR: azureml-contrib-notebook 1.19.0 has requirement nbconvert<6, but you'll have nbconvert 6.0.7 which is incompatible.
ERROR: azure-cli-core 2.16.0 has requirement msal~=1.0.0, but you'll have msal 1.6.0 which is incompatible.
ERROR: azure-cli-core 2.16.0 has requirement msal-extensions~=0.1.3, but you'll have msal-extensions 0.2.2 which is incompatible.
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 20.0.1
    Uninstalling pyOpenSSL-20.0.1:
      Successfully uninstalled pyOpenSSL-20.0.1


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.19.0 to work with MLWorkspace


In [4]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '3de5d625-9504-483a-bb05-8cc0fc7fb6f0'
resource_group = 'DeepLearning'
workspace_name = 'MLWorkspace'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='DocumentClassificationDataset')
dataset.download(target_path='.', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/computeinstanceforrnd/code/preprocesstext_final_V1.csv']

In [5]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="document_classification")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
Corpus = pd.read_csv('/mnt/batch/tasks/shared/LS_root/mounts/clusters/computeinstanceforrnd/code/preprocesstext_final_V1.csv',encoding='latin-1')

# Split the model into Train and Test Data set
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['Text'],Corpus['Class'],train_size=0.8)

# Label encode the target variable  - This is done to transform Categorical data of string type in the data set into numerical values
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# Vectorize the words by using TF-IDF Vectorizer - This is done to find how important a word in document is in comaprison to the corpus
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['Text'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print('SVM Accuracy Score -> ',accuracy_score(predictions_SVM, Test_Y)*100)
run.log('Accuracy', accuracy_score(predictions_SVM, Test_Y)*100)

# Save the trained model
model_file = 'documentclassification_model.pkl'
joblib.dump(value=SVM, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

#save vectorizer
vectorizer_file = 'documentclassification_vectorizer.pkl'
joblib.dump(value = Tfidf_vect, filename=vectorizer_file)
run.upload_file(name = 'outputs/' + vectorizer_file, path_or_stream = './' + vectorizer_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/documentclassification_model.pkl', model_name='documentclassification_model',
                   tags={'Training context':'Inline Training'},
                   properties={'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Using TensorFlow backend.


Starting experiment: document_classification
Loading Data...
SVM Accuracy Score ->  88.18690358670615
Model trained and registered.


In [6]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

documentclassification_model version: 3
	 Training context : Inline Training
	 Accuracy : 88.18690358670615


documentclassification_vectorizer version: 1
	 Training context : Inline Training


documentclassification_model version: 2
	 Training context : Inline Training
	 Accuracy : 88.18690358670615


documentclassification_model version: 1
	 Training context : Inline Training
	 Accuracy : 87.2326423165515




In [7]:
model = ws.models['documentclassification_model']
print(model.name, 'version', model.version)

documentclassification_model version 3


In [8]:
import os

folder_name = 'documentclassification_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_documentclassification.py")

documentclassification_service folder created.


In [9]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('documentclassification_model')
    model = joblib.load(model_path)
    
# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction 
    class_dict = {0: 'letter', 1: 'form', 2: 'invoice', 3: 'advertisement', 4:'email'}
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(class_dict[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./documentclassification_service/score_documentclassification.py


In [10]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"docclassification_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./documentclassification_service/docclassification_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [12]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "documentclassification-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [28]:
print(service.get_logs())

None


In [14]:
for webservice_name in ws.webservices:
    print(webservice_name)

documentclassification-service


In [39]:
import json

x_new = ['This is an admission application form']
print ('Data: {}'.format(x_new))
vect = Tfidf_vect.transform(x_new).toarray()

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": vect.tolist()})
input_json = bytes(input_json, encoding = 'utf8')

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print('The Predicted document is : {}'.format(predicted_classes[0]))

Data: ['This is an admission application form']
The Predicted document is : form


In [37]:
service.get_logs()

'2021-01-10T15:19:21,141467700+00:00 - gunicorn/run \n2021-01-10T15:19:21,155152200+00:00 - rsyslog/run \n2021-01-10T15:19:21,153623100+00:00 - iot-server/run \n2021-01-10T15:19:21,162507600+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [23]:
endpoint = service.scoring_uri
print(endpoint)

http://282f74f4-5bcc-47db-ade9-309c46afaef6.eastus2.azurecontainer.io/score


In [36]:
import json
import requests

x_new = [['This is an admission application form'],['New York is the best city for world-class tutors']]
for i in range(len(x_new)):
    print ('Data: {}'.format(x_new[i]))
    vect = Tfidf_vect.transform(x_new[i]).toarray()

    # Convert the array to a serializable list in a JSON document
    input_json = json.dumps({"data": vect.tolist()})

    # Set the content type
    headers = { 'Content-Type':'application/json' }

    predictions = requests.post(endpoint, input_json, headers = headers)
    predicted_classes = json.loads(predictions.json())
    print ('The Predicted document is : {}'.format(predicted_classes[0]))

Data: ['This is an admission application form']
The Predicted document is : form
Data: ['New York is the best city for world-class tutors']
The Predicted document is : advertisement


In [40]:
service.delete()
print ('Service deleted.')

Service deleted.
